In [2]:
import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50


import os

In [3]:
# Convoluted Base MODEL

conv_base = ResNet50(weights='imagenet',include_top=False, input_shape=(224,224 , 3))

print(conv_base.summary())

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

print(model.summary())


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 126,354,309
Trainable params: 126,301,189
Non-trainable params: 53,120
_____________________________________

In [4]:
for layer in conv_base.layers[:]:
   layer.trainable = False

print('conv_base is now NOT trainable')

conv_base is now NOT trainable


In [5]:
for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)
print(model.summary())

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [6]:
train_dir ='F:/yasmin/roi/train_res'
test_dir = 'F:/yasmin/roi/test_res'

In [7]:
train_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input,
    rotation_range=25.,
    channel_shift_range=20.,
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
        )

test_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input
    )


In [8]:
train_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=64,subset='training')
validation_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=64,subset='validation')
test_batches = test_datagen.flow_from_directory(test_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=1)


Found 5052 images belonging to 5 classes.
Found 1262 images belonging to 5 classes.
Found 1344 images belonging to 5 classes.


In [9]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/roi/new_model_check1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [11]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=3, callbacks=callbacks_list, verbose=1)



Epoch 1/3
78/78 [==============================] - ETA: 0s - loss: 10.2056 - accuracy: 0.4647
Epoch 00001: val_accuracy improved from -inf to 0.60855, saving model to F:/yasmin/roi/new_model_check1.h5
78/78 [==============================] - 896s 11s/step - loss: 10.2056 - accuracy: 0.4647 - val_loss: 3.1700 - val_accuracy: 0.6086
Epoch 2/3
78/78 [==============================] - ETA: 0s - loss: 2.9402 - accuracy: 0.5533
Epoch 00002: val_accuracy improved from 0.60855 to 0.61266, saving model to F:/yasmin/roi/new_model_check1.h5
78/78 [==============================] - 903s 12s/step - loss: 2.9402 - accuracy: 0.5533 - val_loss: 2.5765 - val_accuracy: 0.6127
Epoch 3/3
78/78 [==============================] - ETA: 0s - loss: 2.4183 - accuracy: 0.5814
Epoch 00003: val_accuracy improved from 0.61266 to 0.63322, saving model to F:/yasmin/roi/new_model_check1.h5
78/78 [==============================] - 870s 11s/step - loss: 2.4183 - accuracy: 0.5814 - val_loss: 2.2472 - val_accuracy: 0.6332

In [12]:
model.save('F:/yasmin/roi/model_roi_save_s1.h5')

In [13]:
# Make last block of the conv_base trainable:

for layer in conv_base.layers[:130]:
   layer.trainable = False
for layer in conv_base.layers[130:]:
   layer.trainable = True

print('Last block of the conv_base is now trainable')

Last block of the conv_base is now trainable


In [14]:
for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)


0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [15]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/roi/new_model_check2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [17]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=10, callbacks=callbacks_list, verbose=1)



Epoch 1/10
78/78 [==============================] - ETA: 0s - loss: 1.6139 - accuracy: 0.6309 
Epoch 00001: val_accuracy improved from -inf to 0.66941, saving model to F:/yasmin/roi/new_model_check2.h5
78/78 [==============================] - 1514s 19s/step - loss: 1.6139 - accuracy: 0.6309 - val_loss: 1.4220 - val_accuracy: 0.6694
Epoch 2/10
78/78 [==============================] - ETA: 0s - loss: 1.0554 - accuracy: 0.6953 
Epoch 00002: val_accuracy improved from 0.66941 to 0.71628, saving model to F:/yasmin/roi/new_model_check2.h5
78/78 [==============================] - 1559s 20s/step - loss: 1.0554 - accuracy: 0.6953 - val_loss: 1.0879 - val_accuracy: 0.7163
Epoch 3/10
78/78 [==============================] - ETA: 0s - loss: 0.9388 - accuracy: 0.7275 
Epoch 00003: val_accuracy improved from 0.71628 to 0.73109, saving model to F:/yasmin/roi/new_model_check2.h5
78/78 [==============================] - 1513s 19s/step - loss: 0.9388 - accuracy: 0.7275 - val_loss: 1.1237 - val_accuracy:

In [18]:
model.save('F:/yasmin/roi/model_roi_save_s2.h5')

In [19]:
for layer in conv_base.layers[:]:
   layer.trainable = True

print('conv_base is now NOT trainable')

conv_base is now NOT trainable


In [20]:
for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)
print(model.summary())

0 input_1 True
1 conv1_pad True
2 conv1_conv True
3 conv1_bn True
4 conv1_relu True
5 pool1_pad True
6 pool1_pool True
7 conv2_block1_1_conv True
8 conv2_block1_1_bn True
9 conv2_block1_1_relu True
10 conv2_block1_2_conv True
11 conv2_block1_2_bn True
12 conv2_block1_2_relu True
13 conv2_block1_0_conv True
14 conv2_block1_3_conv True
15 conv2_block1_0_bn True
16 conv2_block1_3_bn True
17 conv2_block1_add True
18 conv2_block1_out True
19 conv2_block2_1_conv True
20 conv2_block2_1_bn True
21 conv2_block2_1_relu True
22 conv2_block2_2_conv True
23 conv2_block2_2_bn True
24 conv2_block2_2_relu True
25 conv2_block2_3_conv True
26 conv2_block2_3_bn True
27 conv2_block2_add True
28 conv2_block2_out True
29 conv2_block3_1_conv True
30 conv2_block3_1_bn True
31 conv2_block3_1_relu True
32 conv2_block3_2_conv True
33 conv2_block3_2_bn True
34 conv2_block3_2_relu True
35 conv2_block3_3_conv True
36 conv2_block3_3_bn True
37 conv2_block3_add True
38 conv2_block3_out True
39 conv3_block1_1_conv Tru

In [21]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(lr=.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [22]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/roi/new_model_check3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=37, callbacks=callbacks_list, verbose=1)


Epoch 1/37
 8/78 [==>...........................] - ETA: 14:31:44 - loss: 1.4042 - accuracy: 0.6484

In [ ]:
model.save('F:/yasmin/roi/model_roi_save_s3.h5')